## **Multiclass Classification using ELMs and kernels** 

## **Objective**

This tutorial demonstrates a multiclass classification using extreme learning machine (ELM) and kernel method. First, you implement ELM by using the least-square method and observe its training and testing accuracy behavior. Second, you compare it with the kernel method when using a linear kernel and investigate the importance of random weights. 

Note: the Bonus exercises are not mandatory and will not be used as a grading criterion. However, interested students are encouraged to solve them to have better understanding.

## **Homeworks**

In this notebook, read the code carefully and complete it to do the following:

1. Complete incomplete sections of the code to build an extreme learning machine (ELM) for different cases.
2. Observe validation accuracy of ELM versus different values of $\lambda = 10^{-2}, 10^{-1}, 10^{0}, \cdots, 10^{3}$ in a single figure, and tune these hyperparameters using grid search. Also, tune $n$ in the same manner.
3. What is the best value for the hyperparameters?
5. Find the testing accuracy of the kernel method when using linear and  Gaussian kernels on the data.
6. *Bonus question*: Can you provide an explanation of why the ELM with Gaussian kernels perform better than the linear kernel case?

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.io import loadmat
import torch
from torch import nn
import matplotlib.pyplot as plt

## **Definition of the required functions**

**About the dataset:** Here, we define some of the functions that we are going to use later in this example. In this example, we use the Vowel dataset which contains 11 different vowels in English as described below. The speakers are indexed by integers 0-89. (Actually, there are fifteen individual speakers, each saying each vowel six times.) For each utterance, there are ten floating-point input values, with array indices 0-9. The problem is to train the network as well as possible using only on data from "speakers" 0-47, and then to test the network on speakers 48-89, reporting the number of correct classifications in the test set. For more information regarding the Vowel dataset refer to [this page](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Vowel+Recognition+-+Deterding+Data)).

**Brief overview of tasks:**
First, you implement ELM by using regularized least-square method and observe its the training and testing accuracy. Let us use a matrix notation for convenience and note the training samples by $\mathbf{X}=[\mathbf{x}_1, \cdots, \mathbf{x}_J] \in \mathbb{R}^{P \times J}$ and their corresponding targets by $\mathbf{T}=[\mathbf{t}_1, \cdots, \mathbf{t}_J] \in \mathbb{R}^{Q \times J}$. Here, $P$ is the input samples dimension, $Q$ is the target dimension (number of classes), and $J$ is the number of training samples. The training of regularized ELM can be written as 
\begin{align}
    \mathbf{W}^{\star} = \underset{\mathbf{W}}{\arg\min} ||\mathbf{T} - \mathbf{W} \mathbf{Y}||_F^2 + \lambda ||\mathbf{W}||_F^2,
\end{align}
where $\mathbf{Y} = \mathbf{g}(\mathbf{R}\mathbf{X}+\mathbf{B}) \in \mathbb{R}^{n \times J}$ is the output of the activation function $\mathbf{g}$ and $n$ is the number of hidden neurons. Note that the weight matrix $\mathbf{R} \in \mathbb{R}^{n \times P}$ and the bias matrix $\mathbf{B}=[\mathbf{b},\cdots,\mathbf{b}] \in \mathbb{R}^{n \times J}$ are chosen as an instance of a random distribution. This minimization problem has the following closed-form solution
\begin{align}
    \mathbf{W}^{\star} = \mathbf{T} \mathbf{Y}'(\mathbf{Y}\mathbf{Y}' + \lambda \mathbf{I})^{-1},
\end{align}
Where $\mathbf{I} \in \mathbb{R}^{n \times n}$ is the identity matrix. The notation $\mathbf{Y}'$ denotes the transpose of the matrix $\mathbf{Y}$. Now we define a member method ${\texttt{LS(Y_train, T_train, lam)}}$ as the least-square function according to the above derivations. We also define ${\texttt{calculate_accuracy}}$ function to measure the classification accuracy.


In [ ]:
# Define the function to load the dataset
def prepare_dataset(dataset_location="./Vowel.mat"):
    """
    An example of how the dataset looks in practice:
    +-------+--------+-------+---------+
        | vowel |  word  | vowel |  word   | 
        +-------+--------+-------+---------+
        |  i    |  heed  |  O    |  hod    |
        |  I    |  hid   |  C:   |  hoard  |
        |  E    |  head  |  U    |  hood   |
        |  A    |  had   |  u:   |  who'd  |
        |  a:   |  hard  |  3:   |  heard  |
        |  Y    |  hud   |       |         |
        +-------+--------+-------+---------+
    """
    X = loadmat(dataset_location)["featureMat"]
    Y = loadmat(dataset_location)["labelMat"]
    X_train, X_test = X[:, :528].astype(np.float32), X[:, 528:].astype(np.float32)
    Y_train, Y_test = Y[:, :528].astype(np.float32), Y[:, 528:].astype(np.float32) 
    return X_train, X_test, Y_train, Y_test

## **Training ELM by using least-square method**

We set the number of hidden neurons $n$ and the regularization hyperparameter $\lambda$, and construct a single layer neural network.

### Define a function to calculate the accuracy

In [ ]:
def calculate_accuracy(S, T):
    # S: predictions
    # T: given targets (ground truth)
    Y = np.argmax(S, axis=0)
    T = np.argmax(T, axis=0)
    accuracy = np.sum([Y == T]) / Y.shape[0] # This computes a classification accuracy
    return accuracy

### Define a class for an Extreme learning machine network

- Complete the following class definition for defining an extreme learning machine

In [ ]:
class ELM(object):
    """
    This defines a class for the extreme learning machine (ELM)
    """
    def __init__(self, n, lam, P):
        self.n = n # Number of hidden nodes 
        self.lam = lam # Initialize the hyperparameter lambda, which is to be tuned
        self.Ri = self.initialise_random_matrix(M=n, N=P) # Initialize the random matrix 
        self.b = self.initialise_random_matrix(M=n, N=1) # Initialize the bias, also randomly
        self.W_ls = None # Initially set to None, because we compute it using least squares
    
    def activation_function(self, Z):
        # Here, we basically implement a ReLU( ) activation function
        return np.maximum(0, Z)

    def initialise_random_matrix(self, M, N):
        """
        Initialize a random matrix that consists of uniformly 
        distributed values in the range [-1, 1]. Matrix should be 
        of shape (M, N)
        """
        # Add code here
        # ...
        return None # Fix this return statement


    def LS(self, Y, T):
        """Solve the optimization problem as regularized least-squares"""
        P = Y.shape[0] # Dimensionality of the input
        # Implement the equation T*Y'*inv(Y*Y' + lambda*I) as the least squares solution
        W_ls = # ... Add code here
        return W_ls
    
    def train(self, X, T):
        """
        Train the ELM network. Note that here training indicates simply calculating
        the matrix W_ls. Since, we have a closed form expression, this amounts to only
        calling the relevant method that we have wrote earlier. 
        """
        N = X.shape[1] # Number of samples in the data X
        
        # Pass the input through the linear transform using the input
        Zi = np.dot(self.Ri, X) + np.tile(self.b,(1,N))
        
        # Obtain the value of Y_i by applying the activation function on Z_i 
        Yi = # ... Add code here 
        
        # Obtain the least squares matrix W_ls
        self.W_ls = self.LS(Y=Yi, T=T)
    
    def evaluate(self, X):
        """
        This function provides predictions using the ELM network, provided that
        the network has been already trained, i.e. the train( ) method has been executed
        """
        N = X.shape[1] # Number of samples in the data X
        
        # Pass the input through the linear transform using the input
        Zi = np.dot(self.Ri, X) + np.tile(self.b,(1,N))
        
        # Obtain the value of Y_i by applying the activation function on Z_i 
        Yi = self.activation_function(Z=Zi)
        
        # Get the prediction using the W_ls
        T_hat = # ... # Add code here 
        return T_hat
        

In [ ]:
def train_ELM(X_train, T_train, X_val=None, T_val=None, n=2000, lam=1e2):
    """
    This function trains and evaluates the ELM by instantiating the ELM class,
    train the network and then evaluate both the training and the validation
    data
    """
    elm_net = ELM(n=n, lam=lam, P=X_train.shape[0])
    elm_net.train(X=X_train, T=T_train)
    T_hat_train = elm_net.evaluate(X=X_train)
    T_hat_val = elm_net.evaluate(X=X_val)
    acc_train = calculate_accuracy(T_hat_train, T_train)
    acc_val = calculate_accuracy(T_hat_val, T_val)
    
    return acc_train, acc_val, elm_net

### Get an initial estimate of accuracy from an ELM

In [ ]:
# Get the dataset
X_train, X_test, T_train, T_test = prepare_dataset(dataset_location="./Vowel.mat")

In [ ]:
P= X_train.shape[0] # Input features
Q = T_train.shape[0] # Output label size
N_train = X_train.shape[1] # Number of samples in training data
N_test = X_test.shape[1] # Number of samples in test data

In [ ]:
print(P, Q, N_train, N_test)

In [ ]:
def split_tr_val_data(X, T, train_val_split=0.8):
    """
    Splits the data randomly into training and validation data in the ratio 80:20
    """
    N = X.shape[1]
    indices = np.random.permutation(N)
    num_train_samples = int(train_val_split * len(indices))
    num_val_samples = len(indices) - num_train_samples
    train_indices = indices[:num_train_samples]
    val_indices = indices[num_train_samples:]
    X_train, T_train = X[:, train_indices], T[:, train_indices]
    X_val, T_val = X[:, val_indices], T[:, val_indices]
    print(X_train.shape, T_train.shape, X_val.shape, T_val.shape)
    return X_train, T_train, X_val, T_val

In [ ]:
tr_to_val_split = 0.8
X_train, T_train, X_val, T_val = split_tr_val_data(X=X_train, T=T_train, train_val_split=tr_to_val_split)

### Executing this cell to train the ELM using a randomly chosen $\lambda$. 

We note that executing this cell seeral times can lead to differing results, but in general the classification accuracy on average is poor. 

In [ ]:
acc_train_star, acc_val_star, elm_net_star = train_ELM(X_train, T_train, X_val, T_val, n=200, lam=1e-14)
T_hat_test = elm_net_star.evaluate(X=X_test)
acc_test_star = calculate_accuracy(T_hat_test, T_test)
print("Lambda: {}, Acc_train: {}, Acc_val: {}, Acc_test: {}".format(1e-2, 
                                                                  acc_train_star, 
                                                                  acc_val_star, 
                                                                  acc_test_star))

## Grid search CV: Sweeping over a set of parameters $\lambda$ to find the ELM with most suited architecture

- We reimplement an `ELM_estimator` by subclassing from `BaseEstimator``
- This kind of reimplementation is helpful for utilising the grid search CV library of `sklearn`.
- Mainly restructuring the train and evaluate functions as: `fit( )`, `predict( )` and `score()` function. 

In [ ]:
from sklearn.base import BaseEstimator

class ELM_estimator(BaseEstimator):
    
    def __init__(self, n=2000, lam=1e-5, P=10):
        super().__init__()
        self.n = n # Number of hidden nodes 
        self.P = P # Input dimensionality
        self.lam = lam # Initialize the hyperparameter lambda, which is to be tuned
        self.Ri = self.initialise_random_matrix(M=self.n, N=self.P) # Initialize the random matrix 
        self.b = self.initialise_random_matrix(M=n, N=1) # Initialize the bias, also randomly
        self.W_ls = None # Initially set to None, because we compute it using
        
    def activation_function(self, Z):
         # Here, we basically implement a ReLU( ) activation function
        return np.maximum(0, Z)

    def initialise_random_matrix(self, M, N):
        """
        Initialize a random matrix that consists of uniformly 
        distributed values in the range [-1, 1]. Matrix should be 
        of shape (M, N)
        """
        # Add code here
        # ...
        return None # Fix this return statement

    def LS(self, Y, T):
        """Solve the optimization problem as regularized least-squares"""
        P = Y.shape[0] # Dimensionality of the input
        # Implement the equation T*Y'*inv(Y*Y' + lambda*I) as the least squares solution
        W_ls = # ... Add code here
        return W_ls
    
    def fit(self, X, T):
        """
        Train the ELM network. Note that here training indicates simply calculating
        the matrix W_ls. Since, we have a closed form expression, this amounts to only
        calling the relevant method that we have wrote earlier. 
        """
        X, T = X.T, T.T
        N = X.shape[1]
        
        # Pass the input through the linear transform using the input
        Zi = np.dot(self.Ri, X) + np.tile(self.b,(1,N))
        
        # Obtain the value of Y_i by applying the activation function on Z_i 
        Yi = # ... Add code here 
        
        self.W_ls = self.LS(Y=Yi, T=T)
        return self
        #T_hat = np.dot(self.W_ls, Yi)
        #return T_hat
    
    def predict(self, X):
        """
        This function provides predictions using the ELM network, provided that
        the network has been already trained, i.e. the train( ) method has been executed
        """
        X = X.T
        N = X.shape[1]
        # Pass the input through the linear transform using the input
        Zi = np.dot(self.Ri, X) + np.tile(self.b,(1,N))
        
        # Obtain the value of Y_i by applying the activation function on Z_i 
        Yi = # ... Add code here 
        T_hat = np.dot(self.W_ls, Yi)
        return T_hat
    
    def score(self, X, T):
        """
        This function is similar to the calculate_accuracy function 
        """
        T = np.transpose(T)
        T_hat = self.predict(X)
        Y = np.argmax(T_hat, axis=0)
        T = np.argmax(T, axis=0)
        
        accuracy = np.sum([Y == T]) / Y.shape[0]
        return accuracy
        

In [ ]:
# You can set these parameter dictionaries as you like to search over a grid of parameters
param_grid = {'lam': np.logspace(-2, 3, 6).tolist(),  # Parameter grid for lambda
              'n': np.linspace(10, 500, 11).astype(np.int16).tolist() # Number of nodes 
             } 

In [ ]:
from sklearn.model_selection import GridSearchCV

### Implementing Grid search and k-fold cross validation.

Here, we consider $K=5$ folds as default. 

- Look up the use of `sklearn.model_selection.GridSearchCV`
- Using `sklearn.model_selection.GridSearchCV` tune the hyperparameters `lam, n`. Use options:
    - 5 cross validation folds
    - refit in case of a new fold
    - Choose an appropriate verbosity option to display progress, e.g. 3
    - If you have multiple cores on your machine, you can parallelize using `n_jobs`
- Display the best hyperparameters through grid search. 

In [ ]:
# Initialize an object of a GridSearchCV class with appropriate options
# Add code here
grid = # ... # 

# Get the dataset
X_train_all, X_test, T_train_all, T_test = prepare_dataset(dataset_location="./Vowel.mat")

# fitting the model for grid search for the data X_train_all, T_train_all using the fit( ) function.
# Add code here
# ...

In [ ]:
# print best parameter after tuning 
# Add code here (HINT: Check the documentation for GridSearchCV class)
# ...

In [ ]:
n_arr = np.unique(grid.cv_results_['param_n'].data)
np.array([[m, p['lam']] for p,m in zip(grid.cv_results_['params'],grid.cv_results_['mean_test_score']) if p['n'] == 20])
FontSize = 30
#csfont = {'fontname':'serif'}
plt.rc( 'font', size=30, family="Times" ) 
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
#plt.plot(lam_arr, val_accuracy_lists, 'r-', label="Validation Accuracy")
#plt.plot(lam_arr, train_accuracy_lists, 'b-', label="Train Accuracy")
for i, n in enumerate(n_arr):
    mean_score_lambda_arr = np.array([[m, p['lam']] for p,m in zip(grid.cv_results_['params'],grid.cv_results_['mean_test_score']) if p['n'] == n])
    plt.plot(mean_score_lambda_arr[:,1], mean_score_lambda_arr[:,0],  'D-', label="Validation Accuracy for n={}".format(n), linewidth=2)

ax.set_xscale('log')
plt.grid()
plt.xticks(fontsize=FontSize)
plt.yticks(fontsize=FontSize)
plt.tight_layout()
plt.legend(loc='best',fontsize=FontSize)
plt.xlabel("Hyperparameter lambda", fontsize=FontSize)
plt.ylabel("Classification Accuracy", fontsize=FontSize)
plt.title("ELM performance on Vowel", loc='center', fontsize=FontSize)
plt.show()

## Get the accuracy results on test data

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = np.argmax(grid.predict(X_test.T) , axis=0).reshape((-1, 1))
# print classification report 
print(classification_report(np.argmax(T_test, axis=0), grid_predictions)) 

In [ ]:
print(grid_predictions.shape, T_test.shape)

## Using kernels with ELMs

### **Training ELM by using linear kernel**

We set the regularization hyperparameter alpha, and construct the kernel matrix for training and test data accordingly. In particular, the solution of the kernel method for a pair of $(\mathbf{y}_j,\mathbf{t}_j)$ can be written as 
\begin{align}
    \hat{\mathbf{t}}_j = \mathbf{T} (K + \alpha \mathbf{I})^{-1} k(\mathbf{y}_j),
\end{align}
where $\mathbf{K} \in \mathbb{R}^{J \times J}$ is the kernel matrix with elements $k_{ij}=\mathbf{k}(\mathbf{y}_i)^{'}\mathbf{k}(\mathbf{y}_j)$ for a given kernel $\mathbf{k}(\cdot)$. Note that $k(\mathbf{y}_j) \in \mathbb{R}^{J}$  is the product of sample $\mathbf{y}_j$ and training set with elements $k_{j}=\mathbf{k}(\mathbf{y}_i)^{'}\mathbf{k}(\mathbf{y}_j), \forall i = 1, \cdots, J$. In matrix form, we can write
\begin{align}
    \hat{\mathbf{T}}_{\text{test}} = \mathbf{T} (K + \alpha \mathbf{I})^{-1} K_{\text{test}},
\end{align}
where $K_{\text{test}} \in \mathbb{R}^{J \times J_{\text{test}}}$ is the kernel matrix between train and test samples for a given kernel $\mathbf{k}(\cdot)$. Now we apply a linear kernel $\mathbf{k}(\cdot)$ on the feature vectors of ELM in $\mathbf{Y}$ to predict the target.

- Complete the code to define the linear kernel
- Complete the code to define the RBF kernel
- Complete the code to compute $\mathbf{W}_{ker} = \mathbf{T} (K + \alpha \mathbf{I})^{-1}$

**NOTE:** We expect the performance of the linear kernel to be roughly similar to what we obtained earlier using the ELM without the kernel.

In [ ]:
class ELM_with_kernel(object):
    
    def __init__(self, n, alpha, P, kernel_type="linear"):
        self.n = n
        self.alpha = alpha
        self.Ri = self.initialise_random_matrix(M=n, N=P)
        self.b = self.initialise_random_matrix(M=n, N=1)
        self.W_ker = None
        self.kernel_type = kernel_type
        
    def linear_kernel(self, X, Y):
        """
        Define a linear kernel K(x, y) = x'*y
        """
        # Add code here
        # ...
        return None
        
    def rbf_kernel(self, X, Y):
        """
        Defines a RBF kernel K(x, y)
        """
        N1 = X.shape[1]
        N2 = Y.shape[1]
        n1sq = np.sum(np.square(X),axis=0)
        n2sq = np.sum(np.square(Y),axis=0)
        D = np.tile(n1sq, (N2, 1)).T + np.tile(n2sq, (N1, 1)) - 2 * np.dot(X.T, Y)  # the element i,j of this matrix are nothing but the individual distance of samples x_i and y_j meaning ||x_i - y_j||_2^2
        C = np.tile(n1sq, (N1, 1)).T + np.tile(n1sq, (N1, 1)) - 2 * np.dot(X.T, X)  # the element i,j of this matrix are nothing but the individual distance of samples x_i and x_j meaning ||x_i - x_j||_2^2
        temp = np.sum(C)/(N1*N1)    # temp represent the average distance of samples of X from each other. Note that C is zero on the diagonal.
        # Add code here to return the value of the RBF kernel using the variables
        K = # ...     # note that in this way we don't need to tune the value of temp
        return K
    
    def activation_function(self, Z):
        # Here, we basically implement a ReLU( ) activation function
        return np.maximum(0, Z)

    def initialise_random_matrix(self, M, N):
        """
        Initialize a random matrix that consists of uniformly 
        distributed values in the range [-1, 1]. Matrix should be 
        of shape (M, N)
        """
        # Add code here
        # ...
        return None # Fix this return statement
    
    def train(self, X, T):
        
        P = X.shape[0]
        N = X.shape[1]
        Zi = np.dot(self.Ri, X) + np.tile(self.b,(1,N))
        Yi = self.activation_function(Z=Zi)
        if self.kernel_type == "linear":
            self.W_ker = # ... # Add code to compute W_ker using a linear kernel (in float32 format)
        elif self.kernel_type == "gaussian":
            self.W_ker = # ... # Add code to compute W_ker using a RBF kernel (in float32 format)
    
    def evaluate(self, X_train, X_test):
        
        N_train = X_train.shape[1]
        N_test = X_test.shape[1]
        
        Zi_train = np.dot(self.Ri, X_train) + np.tile(self.b,(1,N_train))
        Yi_train = self.activation_function(Z=Zi_train)
        
        Zi_test = np.dot(self.Ri, X_test) + np.tile(self.b,(1,N_test))
        Yi_test = self.activation_function(Z=Zi_test)
        
        if self.kernel_type == "linear":
            K_test = self.linear_kernel(Yi_train, Yi_test)
        elif self.kernel_type == "gaussian":
            K_test = self.rbf_kernel(Yi_train, Yi_test)
        
        T_hat = np.dot(self.W_ker, K_test)
        return T_hat

In [ ]:
def train_ELM_with_kernel(X_train, T_train, X_val=None, T_val=None, n=2000, alpha=1e2, kernel_type="linear"):
    
    elm_net = ELM_with_kernel(n=n, alpha=alpha, P=X_train.shape[0], kernel_type=kernel_type)
    elm_net.train(X=X_train, T=T_train)
    T_hat_train = elm_net.evaluate(X_train=X_train, X_test=X_train)
    T_hat_val = elm_net.evaluate(X_train=X_train, X_test=X_val)
    acc_train = calculate_accuracy(T_hat_train, T_train)
    acc_val = calculate_accuracy(T_hat_val, T_val)
    
    return acc_train, acc_val, elm_net

In [ ]:
grid.best_params_

### Train an ELM using Linear kernel, using the same parameters optimized earlier for $\lambda, n$

- Complete the code to train the ELM using a Linear kernel
- Complete the code to evaluate using the ELM trained using linear kernel on the test data

In [ ]:
acc_train_linear_kernel_star, acc_val_linear_kernel_star, elm_net_linear_kernel_star = # ... # Add code here 
T_hat_test_linear_kernel = # ... # Add code here
acc_test_linear_kernel_star = calculate_accuracy(T_hat_test_linear_kernel, T_test)
print("Chosen Lambda: {}, Acc_train: {}, Acc_val: {}, Acc_test: {}".format(grid.best_params_['lam'], 
                                                                              acc_train_linear_kernel_star, 
                                                                              acc_val_linear_kernel_star, 
                                                                              acc_test_linear_kernel_star))

### Training with Gaussian / RBF kernels using the same parameters optimized earlier for $\lambda, n$

- Complete the code to train the ELM using a Gaussian kernel
- Complete the code to evaluate using the ELM on the test data

**NOTE:** We expect an improvement in the classification accuracy using the RBF kernel

In [ ]:
acc_train_gaussian_kernel_star, acc_val_gaussian_kernel_star, elm_net_gaussian_kernel_star = # ... # Add code here
T_hat_test_gaussian_kernel = # ... # Add code here
acc_test_gaussian_kernel_star = calculate_accuracy(T_hat_test_gaussian_kernel, T_test)
print("Chosen Lambda: {}, Acc_train: {}, Acc_val: {}, Acc_test: {}".format(grid.best_params_['lam'], 
                                                                              acc_train_gaussian_kernel_star, 
                                                                              acc_val_gaussian_kernel_star, 
                                                                              acc_test_gaussian_kernel_star))

### Bonus Question: Can you think why we get an improvement using a RBF / gaussian kernel?

Write your answer in text in this markdown cell